# InfoHub Class for Connection and Queries (Sync Version)

## `gql`

* Uses `gql` Python package
* Reference: [GQL Documentation](https://gql.readthedocs.io/en/latest/index.html)

## Synchronous Version

* Synchronous transport (`requests`) is used in this version.
* Asynchronous transport (`asyncio`) version will be added soon.

In [1]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

In [2]:
class InfoHubConnection:

    def __init__(self, url, tenantId, headers):
        self.__headers = headers
        self.__tenantId = tenantId
        self.__url = url
        # Select your transport with a defined url endpoint
        self.__transport = RequestsHTTPTransport(url=self.__url, headers=self.__headers)
        # Create a GraphQL client using the defined transport
        self.__client =  Client(transport=self.__transport, fetch_schema_from_transport=True, execute_timeout=60)

    # Get the list of work queues
    def get_work_queues(self):
        """Get the list of work queues
        
        Arguments:
            None
            
        Returns:
            List of Dicts - List of work queues
        """
        query = gql(
        """
        query WorkQueuesList($tenantId: String, $sortBy: String = "priority", $cursorId: String, $pageLength: Int = 50, $sortOrder: SortOrder = DESC) {
            workQueues: businessObjects(
                simpleFilter: {type: WorkQueueDefinition, tenantId: $tenantId}
                cursorParams: {after: $cursorId, first: $pageLength, sort: {fieldPath: $sortBy, order: $sortOrder}}
            ) {
                totalCount
                pageInfo {
                  endCursor
                  hasNextPage
                }
                edges {
                  object {
                    ... on WorkQueueDefinition {
                      name
                      id
                      logicalName
                      counts {
                        currentOpenCount
                        __typename
                      }
                      primaryBusinessObjectTypes
                    }
                  }
               }
            }
        }
        """
        )
        query_variables = {
            "pageLength": 10,
            "sortBy": "priority",
            "sortOrder": "DESC",
            "tenantId": self.__tenantId,
            "cursorId" : ''
        }
        
        workqueues = []
        result = self.__client.execute(query, variable_values=query_variables)
        workqueues.extend(result['workQueues']['edges'])
        query_variables['cursorId'] = result['workQueues']['pageInfo']['endCursor']
        next_page = result['workQueues']['pageInfo']['hasNextPage']
        
        while(next_page):            
            res = self.__client.execute(query, variable_values=query_variables)
            next_page = res['workQueues']['pageInfo']['hasNextPage']
            workqueues.extend(res['workQueues']['edges'])
            query_variables['cursorId'] = res['workQueues']['pageInfo']['endCursor']
        return workqueues
    
    # Get the workitems in progress of a workqueue (given its ID)
    def get_workitems_in_progress(self, workQueueId):
        """Get the workitems in progress of a workqueue
        
        Aruguments:
            workQueueID {string} -- Object ID of the workqueue
            
        Returns:
            List of dicts -- List of workitems 
        """
        query = gql(
        """
        query WorkItemsInProgress($tenantId: String!, $workQueueId: String!, $cursorId: String, $pageLength: Int = 10, $sortBy: String = "id", $sortOrder: SortOrder = DESC) {
            workItemsInProgress: businessObjects(
            hint: {viewId: "graph"}
            simpleFilter: {type: WorkItem, tenantId: $tenantId}
            cursorParams: {first: $pageLength, after: $cursorId, sort: {fieldPath: $sortBy, order: $sortOrder}}
            advancedFilter: {
                AND: [
                    {EQUALS: [{SELECT: "sourceType", type: STRING}, {VALUE: "WorkQueueDefinition", type: STRING}]}, 
                    {EQUALS: [{SELECT: "sourceId", type: STRING}, {VALUE: $workQueueId, type: STRING}]}, 
                    {EXISTS: [{SELECT: "userAssigned", type: STRING}]}, 
                    {OR: [
                        {NOT: [{EQUALS_ANY: [{SELECT: "status", type: STRING}, {VALUES: ["closed", "resolutionPending"], type: STRING}]}]}, 
                        {NOT: {EXISTS: [{SELECT: "status", type: STRING}]}}
                        ]
                    }
                ]
            } ) {
                totalCount
                pageInfo {
                    hasNextPage
                    endCursor
                }
                edges {
                    object {
                        ... on WorkItem {
                            id
                            priority
                            businessObject {
                                id
                                ... on Inventory {
                                    product {
                                        id
                                        partNumber
                                    }
                                    location {
                                        id
                                        locationIdentifier
                                    }
                                    quantity
                                    quantityBelowLowerThreshold
                                    quantityLowerThreshold
                                }
                            }
                        }
                    }
                }
            }
        }
        """
        )

        query_variables = {
            "pageLength": 5,
            "sortBy": "priority",
            "sortOrder": "DESC",
            "tenantId": self.__tenantId,
            "workQueueId": workQueueId,
            "cursorId": ""
        }  
        
        workitems = []
        result = self.__client.execute(query, variable_values=query_variables)
        workitems.extend(result['workItemsInProgress']['edges'])
        query_variables['cursorId'] = result['workItemsInProgress']['pageInfo']['endCursor']
        next_page = result['workItemsInProgress']['pageInfo']['hasNextPage']
        
        while(next_page):            
            res = self.__client.execute(query, variable_values=query_variables)
            next_page = res['workItemsInProgress']['pageInfo']['hasNextPage']
            workitems.extend(res['workItemsInProgress']['edges'])
            query_variables['cursorId'] = res['workItemsInProgress']['pageInfo']['endCursor']
            
        return workitems      
        
    # Get supply plans for an inventory [part-number, location-identifier]
    def get_supplyplans(self, partNumber, locationIdentifier, maxDate):
        """Get supply plans for an inventory [part-number, location-identifier]
        
        Arguments:
            partNumber {string} -- Part number
            locationIdentifier {string} -- Location identifier
            maxDate {string} -- Date in ISO format; Supply plans until this date are retrieved
            
        Returns:
            List of dicts -- List of supply plans
        """
        query = gql(
        """
        query SupplyPlans($tenantId: String, $partNumber: String, $locationIdentifier: String, $maxDate: String, $cursorId: String, $pageLength: Int = 50, $sortBy: String = "id", $sortOrder: SortOrder = DESC) {
            supplyPlan: businessObjects(
                hint: {viewId: "graph"}
                simpleFilter: {tenantId: $tenantId, type: SupplyPlan}
                advancedFilter: {
                    AND: [
                        {EQUALS: [{SELECT: "product.partNumber", type: STRING}, {VALUE: $partNumber, type: STRING}]}, 
                        {EQUALS: [{SELECT: "location.locationIdentifier", type: STRING}, {VALUE: $locationIdentifier, type: STRING}]},
                        {LESS_THAN: [{SELECT: "startDate", type: STRING}, {VALUE: $maxDate, type: STRING}]}
                    ]
                }
                cursorParams: {after: $cursorId, first: $pageLength, sort: {fieldPath: $sortBy, order: $sortOrder}}
            ) {
                totalCount
                pageInfo {
                    endCursor
                    hasNextPage
                }
                edges {
                    object {
                        ... on SupplyPlan {
                            startDate
                            quantity
                            id
                        }
                    }
                }
            }
        }    
        """
        )

        query_variables = {
            "pageLength": 10,
            "sortBy": "startDate",
            "sortOrder": "ASC",
            "tenantId": self.__tenantId,
            "partNumber": partNumber,
            "locationIdentifier": locationIdentifier,
            "maxDate": maxDate,
            "cursorId" : ''
        }
        plans = []
        result = self.__client.execute(query, variable_values=query_variables)
        plans.extend(result['supplyPlan']['edges'])
        query_variables['cursorId'] = result['supplyPlan']['pageInfo']['endCursor']
        next_page = result['supplyPlan']['pageInfo']['hasNextPage']
        
        while(next_page):            
            res = self.__client.execute(query, variable_values=query_variables)
            next_page = res['supplyPlan']['pageInfo']['hasNextPage']
            plans.extend(res['supplyPlan']['edges'])
            query_variables['cursorId'] = res['supplyPlan']['pageInfo']['endCursor']
            
        return plans        
        
    # Get demand plans for an inventory [part-number, location-identifier]
    def get_demandplans(self, partNumber, locationIdentifier, maxDate):
        """Get demand plans for an inventory [part-number, location-identifier]
        
        Arguments:
            partNumber {string} -- Part number
            locationIdentifier {string} -- Location identifier
            maxDate {string} -- Date in ISO format; Demand plans until this date are retrieved
            
        Returns:
            List of dicts -- List of demand plans
        """
        query = gql(
        """
        query DemandPlans($tenantId: String, $partNumber: String, $locationIdentifier: String, $maxDate: String, $cursorId: String, $pageLength: Int = 50, $sortBy: String = "id", $sortOrder: SortOrder = DESC) {
            demandPlan: businessObjects(
                hint: {viewId: "graph"}, 
                simpleFilter: {tenantId: $tenantId, type: DemandPlan}, 
                advancedFilter: {
                    AND: [
                        {EQUALS: [{SELECT: "product.partNumber", type: STRING}, {VALUE: $partNumber, type: STRING}]}, 
                        {EQUALS: [{SELECT: "location.locationIdentifier", type: STRING}, {VALUE: $locationIdentifier, type: STRING}]}, 
                        {LESS_THAN: [{SELECT: "startDate", type: STRING}, {VALUE: $maxDate, type: STRING}]}
                    ]
                }, 
                cursorParams: {after: $cursorId, first: $pageLength, sort: {fieldPath: $sortBy, order: $sortOrder}}
            ) {
                totalCount
                pageInfo {
                    endCursor
                    hasNextPage
                }
                edges {
                    object {
                        ... on DemandPlan {
                            startDate
                            quantity
                            id
                        }
                    }
                }
            }
        }
        """
        )

        query_variables = {
            "pageLength": 10,
            "sortBy": "startDate",
            "sortOrder": "ASC",
            "tenantId": self.__tenantId,
            "partNumber": partNumber,
            "locationIdentifier": locationIdentifier,
            "maxDate": maxDate,
            "cursorId" : ''
        }
        
        plans = []
        result = self.__client.execute(query, variable_values=query_variables)
        plans.extend(result['demandPlan']['edges'])
        query_variables['cursorId'] = result['demandPlan']['pageInfo']['endCursor']
        next_page = result['demandPlan']['pageInfo']['hasNextPage']
        
        while(next_page):            
            res = self.__client.execute(query, variable_values=query_variables)
            next_page = res['demandPlan']['pageInfo']['hasNextPage']
            plans.extend(res['demandPlan']['edges'])
            query_variables['cursorId'] = res['demandPlan']['pageInfo']['endCursor']
            
        return plans
    
    # Upsert Priority Score of a Work Item
    def upsert_workitem_priority(self, workItemId, priority, timestamp):
        """Upsert Priority Score of a Work Item
        
        Arguments:
            workItemId {string} -- Work item object ID
            priority {integer} -- Priority score [0, 100]
            timestamp {string} -- Timestamp in ISO format to record the upsert event
            
        Returns:
            Dict -- Dict consisting of `upsertWorkItem` (boolean) and upsert event `id` (string)
        """
        query = gql(
        """
        mutation($tenantId: String!, $workItemId: ID, $priority: Int, $timestampEventOccurred: String!) {
            upsertWorkItem(workItemEvent: {
                tenantId: $tenantId
                timestampEventOccurred: $timestampEventOccurred
                eventDetails: {
                    businessObject: {
                        id: $workItemId
                        priority: $priority
                    }
                }
            })
        }
        """
        )
        query_variables = {
            "tenantId": self.__tenantId,
            "workItemId": workItemId,
            "priority": priority,
            "timestampEventOccurred": timestamp
        }
        
        result = self.__client.execute(query, variable_values=query_variables)
        return result
    
    # Upsert Supply Plan
    def upsert_supplyplan(self, supplyPlanID, quantity, startDate, timestamp):
        """Upsert supply plan
        
        Arguments:
            supplyPlanID {string} -- Supply plan object ID
            quantity {float} -- Supply quantity > 0.0
            timestamp {string} -- Timestamp in ISO format to record the upsert event
            
        Returns:
            Dict -- Dict consisting of `upsertSupplyPlan` (boolean) and upsert event `id` (string)
        """
        query = gql(
        """
        mutation ($tenantId: String!, $supplyPlanId: ID, $quantity: Float, $startDate: String, $timestampEventOccurred: String!) {
            upsertSupplyPlan(supplyPlanEvent: {
                tenantId: $tenantId, 
                timestampEventOccurred: $timestampEventOccurred, 
                eventDetails: {
                    businessObject: {
                        id: $supplyPlanId, 
                        quantity: $quantity,
                        startDate: $startDate
                    }
                }
            })
        }
        """
        )
        query_variables = {
            "tenantId": self.__tenantId,
            "supplyPlanId": supplyPlanID,
            "quantity": quantity,
            "startDate": startDate,
            "timestampEventOccurred": timestamp
        }
        result = self.__client.execute(query, variable_values=query_variables)
        return result
    
    def upsert_demandplan(self, demandPlanID, startDate, quantity, timestamp):
        """Upsert demand plan
        
        Arguments:
            demandPlanID {string} -- Demand plan object ID
            quantity {float} -- Demand quantity > 0.0
            timestamp {string} -- Timestamp in ISO format to record the upsert event
            
        Returns:
            Dict -- Dict consisting of `upsertDemandPlan` (boolean) and upsert event `id` (string)
        """
        query = gql(
        """
        mutation ($tenantId: String!, $demandPlanId: ID, $quantity: Float, $startDate: String, $timestampEventOccurred: String!) {
            upsertDemandPlan(demandPlanEvent: {
                tenantId: $tenantId, 
                timestampEventOccurred: $timestampEventOccurred, 
                eventDetails: {
                    businessObject: {
                        id: $demandPlanId, 
                        quantity: $quantity,
                        startDate: $startDate
                    }
                }
            })
        }
        """
        )
        query_variables = {
            "tenantId": self.__tenantId,
            "demandPlanId": demandPlanID,
            "startDate": startDate,
            "quantity": quantity,
            "timestampEventOccurred": timestamp
        }
        result = self.__client.execute(query, variable_values=query_variables)
        return result